# Compartmental models for disease spread: Endemic state


## Instructions:

* Run the cell [Packages and Functions](#Packages-and-Functions) first, all the relevant code is there (except, maybe, some plotting routines).


## Notation:

* $S$ and $I$ denote *fractions* of susceptible and infected population.
* $t$ is in units of $1/\gamma$, the inverse of the recovery rate.
* $R_{0}$ represents the basic reproduction number/rate.
* $\nu$ is the ratio between waining immunity rate and recovery rate.


## Index

### [1. SIR and SIRS models - Relevant formulae](#1-SIR-and-SIRS-models---Relevant-Formulae)

### [2. Comparison of SIR and SIRS models](#2-Comparison-of-SIR-and-SIRS-models)

### [3. Bifurcation diagram](#3-Bifurcation-Diagram)

### [Packages and functions](#Packages-and-Functions)

<hr style="border: 1px solid black; width:100%;"></hr>


## 1 SIR and SIRS models - Relevant Formulae

Since for both cases the total population is conserved in time, $\dot{S} + \dot{I} + \dot{R} = 0$, we can treat the system in terms of two variables instead of three through the relation $R = 1 - S - I$. Then, the systems read

\begin{equation}
    \text{SIR model:} \quad
    \left\{
    \begin{aligned}
        \frac{\mathrm{d}S}{\mathrm{d}t} & = -R_{0} S I \\
        \frac{\mathrm{d}I}{\mathrm{d}t} & = (R_{0} S - 1) I 
    \end{aligned}
    \right\} \qquad \qquad 
    \text{SIRS model:} \quad
    \left\{
    \begin{aligned}
        \frac{\mathrm{d}S}{\mathrm{d}t} & =  -R_{0} S I + \nu(1 - S - I) \\
        \frac{\mathrm{d}I}{\mathrm{d}t} & = (R_{0} S - 1) I 
    \end{aligned}
    \right\}
\end{equation}

For the SIR system we always have a **single stable fixed point** given by

\begin{equation}
    S^{*} = I_{0} + S_{0} + \frac{1}{R_{0}} \ln{\left(\frac{S^{*}}{S_{0}}\right)}, \qquad I^{*} = 0.
\end{equation}

For the SIRS model we have two fixed points

\begin{equation}
    P_{1} : S^{*}_{1} = 1, \quad I^{*}_{1} = 0, \qquad \qquad P_{2}: S^{*}_{2} = \frac{1}{R_{0}}, \quad I^{*}_{2} = \frac{\nu(R_{0} - 1)}{R_{0}(\nu - 1)}
\end{equation}

and their stability depends on the value of $R_{0}$

* If $R_{0} < 1$ the **first fixed point is stable and the second one is a saddle node**. However, since the second one lies beyond physical initial conditions ($S(t) \leq 1 \; \forall t$), it is never realized.


* If $R_{0} = 1$ both fixed points “merge” into **one stable point**, corresponding to the expression for the first one.


* If $R_{0} > 1$ the first one becomes a saddle node, only approached if $I_{0} = 0$, while **the second fixed point is stable**.


## 2 Comparison of SIR and SIRS models

### [Back to index](#Index)

In [ ]:
#################################
# Initial values and parameters #
#################################

ν = 0.3  # Ratio: waning immunity / recovery

s0 = 0.9
i0 = 1 - s0

tf = 25.  # Final time for integration.

s_curves = []
i_curves = []
times = []
R0_values = []

for case in [1, 4, 5]

    R0 = get_R0_option(case, ν)
    
    push!(R0_values, R0)

    sir_sol, sirs_sol = get_solutions(s0, i0, R0, ν; tf=tf)
    
    push!(s_curves, sir_sol[1, :])
    push!(s_curves, sirs_sol[1, :])
    
    push!(i_curves, sir_sol[2, :])
    push!(i_curves, sirs_sol[2, :])
    
    (case == 1) && (times = sir_sol.t)
    
end

In [ ]:
line_colors = [1 1 :forestgreen :forestgreen 2 2]

labels = ["\$ $(round(R0_values[1], digits=2))\$", false, 
    "\$ $(round(R0_values[2], digits=2))\$", false, 
    "\$ $(round(R0_values[3], digits=2))\$", false]
labels = reduce(hcat, labels)

# General single-plot attributes
kw =(;
    xlabel="\$ t \\; (1/\\gamma \\; \\mathrm{units}) \$",
    xminorgrid=true,
    yminorgrid=true,
    yminorticks=5,
    xlabelfontsize=15,
    ylabelfontsize=15,  # Default value is 11.
    legendfontsize=14,
    legendtitlefontsize=15,
    tickfontsize=10,
    framestyle=:box,
    dpi=400,
    size=(450, 450)
)

plot1 = plot(times, s_curves, label=false, lc=line_colors, ls=[:solid :dash], lw=2, 
        ylabel="\$ S \$"; kw...)

plot2 = plot(times, i_curves, label=labels, lc=line_colors, ls=[:solid :dash], lw=2, 
        ylabel="\$ I \$", legendtitle="\$ R_{0} \$", legend=:outertopright; kw...)

my_plot = plot(plot1, plot2, layout=grid(1, 2, widths=(4.45/10,5.55/10)), size=(910, 450), 
        left_margin=5Plots.mm, bottom_margin=5Plots.mm)

# savefig(my_plot, "Plots/prob1_b_model_comparison.png")

## 3 Bifurcation Diagram

### [Back to index](#Index)

In [ ]:
ν = 0.4  # Ratio: waning immunity / recovery

R0_min = 0.1
R0_max = 3

R0_range1 = R0_min:0.01:1
R0_range2 = 1:0.01:R0_max

# Second fixed point value for I
i_star2(R0, ν) = ν*(R0 - 1)/(R0*(1 + ν))


# General plot attributes
kw =(;
    xlabel="\$ R_{0} \$",
    ylabel="\$ I^{*} \$",
    xrange=(0, R0_max+0.02),
    yrange=(-0.1, i_star2(R0_max, ν)),
    xminorgrid=true,
    yminorgrid=true,
    xminorticks=5,
    yminorticks=5,
    xlabelfontsize=15,
    ylabelfontsize=15,  # Default value is 11.
    legendfontsize=14,
    legendtitlefontsize=15,
    tickfontsize=10,
    framestyle=:box,
    dpi=400,
    size=(450, 450)
)

bif_diag = plot(R0_range1, i_star2.(R0_range1, ν), ls=:dash, lc=1, lw=2, label=false; kw...)
plot!(R0_range2, i_star2.(R0_range2, ν), ls=:solid, lc=1, lw=2, label=false)
plot!(R0_range1, zeros(length(R0_range1)), ls=:solid, lc=2, lw=2, label=false)
plot!(R0_range2, zeros(length(R0_range2)), ls=:dash, lc=2, lw=2, label=false)

# savefig(bif_diag, "Plots/prob1_b_bif_diagram.png")

## Packages and Functions

### [Back to index](#Index)

In [ ]:
using Roots
using DifferentialEquations
using Plots


###################################
# Definition of SIR & SIRS models #
###################################

function sir!(du, u, R0, t)
    s, i = u
    du[1] = ds = -R0 * s * i
    du[2] = di = R0 * s * i - i
end


function sirs!(du, u, params, t)
    R0, ν = params 
    s, i = u
    du[1] = ds = -R0*s*i + ν*(1 - s - i)
    du[2] = di = (R0*s - 1.)*i
end


########################################
# Wrapper for computing both solutions #
########################################

function get_solutions(s0, i0, R0, ν; t0=0, tf=50., save_steps=0.01)
   
    # Declare ODE Problems
    u0 = [s0, i0]
    params = [R0, ν]
    tspan = (t0, tf)
    sir_prob = ODEProblem(sir!, u0, tspan, R0)
    sirs_prob = ODEProblem(sirs!, u0, tspan, params)
    
    # Solve
    sir_sol = solve(sir_prob, saveat = save_steps)
    sirs_sol = solve(sirs_prob, saveat = save_steps)

    return sir_sol, sirs_sol
end


# Trace and discriminant of the Jacobian matrix at 
# the fixed point S = 1/R0, I = ν(R0 - 1)/R0(1+ν).
τ(R0, ν) = -ν*(R0 + ν)/(1 + ν)
Δ(R0, ν) = round((τ(R0, ν))^2 - 4*ν*(R0 - 1), digits=13)


"""
    get_R0_option(num, ν)

Get an R0 value representative of one of the possible
cases for the stability of the fixed points.

1. First f.p. is a stable node, second is a saddle node.
2. First f.p. is a stable degenerate node, second is a saddle node.
3. Single stable node.
4. First f.p. is a saddle node, second is a stable node.
5. First f.p. is a saddle node, second is a stable spiral.
6. Degenerate stable node.
"""
function get_R0_option(num, ν)
    if num == 1
        R0 = (0.7 != 1-ν) ? 0.7 : 0.8
        
    elseif num == 2
        R0 = 1 - ν
        
    elseif num == 3
        R0 = 1
    
    elseif num == 4
        R0 = ((ν^2 + 4*ν + 2 - 2*(1+ν)^(3/2))/ν + 1)/2

    elseif num == 5
        R0_min = (ν^2 + 4*ν + 2 - 2*(1+ν)^(3/2))/ν
        R0_max = (ν^2 + 4*ν + 2 + 2*(1+ν)^(3/2))/ν

        R0_values = range(R0_min, step=(R0_max - R0_min)/100, length=100)

        _, idx = findmax(sqrt.(-Δ.(R0_values, ν)) ./ (-τ.(R0_values, ν)))

        R0 = R0_values[idx]

    elseif num == 6
        R0 = (ν^2 + 4*ν + 2 - 2*(1+ν)^(3/2))/ν
        
    else
        error("Fo' shizzle, dog")
    end
    
    println("R0 value of $(round(R0, digits=3))")

    return R0
end